# Extension of TensorFlow Lite for Microcontroller
Custom implementation of unsupported TensorFlow operators in TensorFlow Lite for Microcontrollers.

### Configure Defaults

In [1]:
# Define paths to model files
MODELS_DIR = 'models/'

import os
from typing import Callable

if not os.path.exists(MODELS_DIR):
    os.mkdir(MODELS_DIR)

### Setup Environment

In [2]:
# Install dependencies
# !pip install -q tensorflow

# Install xxd if it is not available
# !apt-get update && apt-get -qq install xxd

In [3]:
# Import dependencies
import math
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

# Reproducible results
np.random.seed(1)
tf.random.set_seed(1)

### Generate C Source File Support Function

In [4]:
def generateCSourceFile(function, tensors, quantization, representative_dataset: Callable=None):

    # Configure Defaults
    MODEL_NO_QUANT_TFLITE = MODELS_DIR + function.__name__ + "_no_quant.tflite"
    MODEL_TFLITE = MODELS_DIR + function.__name__ + ".tflite"
    MODEL_TFLITE_MICRO = MODELS_DIR + function.__name__ + ".cc"

    # Convert to a TensorFlow Lite Model
    converter = tf.lite.TFLiteConverter.from_concrete_functions([function.get_concrete_function(*tensors)])
    converter.allow_custom_ops = True
    model_no_quant_tflite = converter.convert()

    # Save the model to disk
    open(MODEL_NO_QUANT_TFLITE, "wb").write(model_no_quant_tflite)

    if quantization:
        # Set the optimization flag
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        # Enforce full-int8 quantization (except inputs/outputs which are always float)
        converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
        # Provide a representative dataset to ensure we quantize correctly
        converter.representative_dataset = representative_dataset

        model_tflite = converter.convert()

        # Save the model to disk
        open(MODEL_TFLITE, "wb").write(model_tflite)
    else:
        MODEL_TFLITE = MODEL_NO_QUANT_TFLITE
    
    # Convert to a C source file
    !xxd -i {MODEL_TFLITE} > {MODEL_TFLITE_MICRO}
    # Update variable names
    REPLACE_TEXT = MODEL_TFLITE.replace('/', '_').replace('.', '_')
    !gsed -i 's/'{REPLACE_TEXT}'/g_model/g' {MODEL_TFLITE_MICRO}

    return MODEL_TFLITE_MICRO

## Squeeze 

### Define TensorFlow Model

In [5]:
@tf.function
def squeeze(input, axis=None):
  return tf.squeeze(input, axis=axis, name="SQUEEZE")

### Squeeze Examples

In [6]:
print("Given a tensor input, this operation returns a tensor of the same type with all dimensions of size 1 removed\n")

input = np.array([[[[[[1]], [[1]], [[1]]]], [[[[1]], [[1]], [[1]]]]]])
print("Number of dimensions:", input.ndim)
print("Shape of tensor:", input.shape)

print("\n")

output_tensor = squeeze(input)
print("Number of dimensions:", output_tensor.ndim)
print("Shape of tensor:", output_tensor.shape)

print("\nIf you don't want to remove all size 1 dimensions, you can remove specific size 1 dimensions by specifying axis\n")

axis = [2, 4]
output_tensor = squeeze(input, axis=axis)
print("Number of dimensions:", output_tensor.ndim)
print("Shape of tensor:", output_tensor.shape)

Given a tensor input, this operation returns a tensor of the same type with all dimensions of size 1 removed

Number of dimensions: 6
Shape of tensor: (1, 2, 1, 3, 1, 1)


Number of dimensions: 2
Shape of tensor: (2, 3)

If you don't want to remove all size 1 dimensions, you can remove specific size 1 dimensions by specifying axis

Number of dimensions: 4
Shape of tensor: (1, 2, 3, 1)


### Convert to TensorFlow Lite for Microcontrollers Model

In [7]:
input = np.array([[[[[[1]], [[1]], [[1]]]], [[[[1]], [[1]], [[1]]]]]])
axis = [2, 4]

function = squeeze
tensors = [input, axis]
quantization = True
def representative_dataset():
     yield([input])

model_tflite_micro = generateCSourceFile(function, tensors, quantization, representative_dataset)

In [8]:
# Print the C source file
!cat {model_tflite_micro}

unsigned char g_model[] = {
  0x1c, 0x00, 0x00, 0x00, 0x54, 0x46, 0x4c, 0x33, 0x00, 0x00, 0x12, 0x00,
  0x1c, 0x00, 0x04, 0x00, 0x08, 0x00, 0x0c, 0x00, 0x10, 0x00, 0x14, 0x00,
  0x00, 0x00, 0x18, 0x00, 0x12, 0x00, 0x00, 0x00, 0x03, 0x00, 0x00, 0x00,
  0x4c, 0x02, 0x00, 0x00, 0xf0, 0x00, 0x00, 0x00, 0xd8, 0x00, 0x00, 0x00,
  0x3c, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00,
  0x0c, 0x00, 0x00, 0x00, 0x08, 0x00, 0x0c, 0x00, 0x04, 0x00, 0x08, 0x00,
  0x08, 0x00, 0x00, 0x00, 0x08, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00,
  0x13, 0x00, 0x00, 0x00, 0x6d, 0x69, 0x6e, 0x5f, 0x72, 0x75, 0x6e, 0x74,
  0x69, 0x6d, 0x65, 0x5f, 0x76, 0x65, 0x72, 0x73, 0x69, 0x6f, 0x6e, 0x00,
  0x05, 0x00, 0x00, 0x00, 0x8c, 0x00, 0x00, 0x00, 0x78, 0x00, 0x00, 0x00,
  0x4c, 0x00, 0x00, 0x00, 0x30, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00,
  0xc6, 0xff, 0xff, 0xff, 0x04, 0x00, 0x00, 0x00, 0x10, 0x00, 0x00, 0x00,
  0x31, 0x2e, 0x35, 0x2e, 0x30, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00,
  0x00, 0x

## One-Hot

### Define TensorFlow Model

In [9]:
@tf.function
def one_hot(indices, depth, on_value=None, off_value=None, axis=None):
  return tf.one_hot(indices, depth, on_value=on_value, off_value=off_value, axis=axis, name="ONE-HOT")

### One-Hot Examples

In [10]:
print("\nThe locations represented by indices in indices take value on_value, while all other locations take value off_value\n")

indices = np.array([0, 2, -1, 1])
depth = 3
on_value = 5.0
off_value = 0.0
axis = -1
print("Indices:", indices)
print("Depth:", depth)

output_tensor = one_hot(indices, depth, on_value=on_value, off_value=off_value, axis=axis)
print("Tensor:", output_tensor)

print("\n")

indices = np.array([[0, 2], [1, -1]])
depth = 3
on_value = 1.0
off_value = 0.0
axis = -1
print("Indices:", indices)
print("Depth:", depth)

output_tensor = one_hot(indices, depth, on_value=on_value, off_value=off_value, axis=axis)
print("Tensor:", output_tensor)


The locations represented by indices in indices take value on_value, while all other locations take value off_value

Indices: [ 0  2 -1  1]
Depth: 3
Tensor: tf.Tensor(
[[5. 0. 0.]
 [0. 0. 5.]
 [0. 0. 0.]
 [0. 5. 0.]], shape=(4, 3), dtype=float32)


Indices: [[ 0  2]
 [ 1 -1]]
Depth: 3
Tensor: tf.Tensor(
[[[1. 0. 0.]
  [0. 0. 1.]]

 [[0. 1. 0.]
  [0. 0. 0.]]], shape=(2, 2, 3), dtype=float32)


### Convert to TensorFlow Lite for Microcontrollers Model

In [11]:
indices = np.array([[0, 2], [1, -1]])
depth = 3
on_value = 1.0
off_value = 0.0
axis = -1

function = one_hot
tensors = [indices, depth, on_value, off_value, axis]
# Quantization not yet supported for op: 'ONE_HOT'
quantization = False

model_tflite_micro = generateCSourceFile(function, tensors, quantization)

In [12]:
# Print the C source file
!cat {model_tflite_micro}

unsigned char g_model[] = {
  0x1c, 0x00, 0x00, 0x00, 0x54, 0x46, 0x4c, 0x33, 0x14, 0x00, 0x20, 0x00,
  0x04, 0x00, 0x08, 0x00, 0x0c, 0x00, 0x10, 0x00, 0x14, 0x00, 0x00, 0x00,
  0x18, 0x00, 0x1c, 0x00, 0x14, 0x00, 0x00, 0x00, 0x03, 0x00, 0x00, 0x00,
  0x40, 0x00, 0x00, 0x00, 0x34, 0x00, 0x00, 0x00, 0x94, 0x00, 0x00, 0x00,
  0x0c, 0x00, 0x00, 0x00, 0x3c, 0x00, 0x00, 0x00, 0x34, 0x00, 0x00, 0x00,
  0x07, 0x00, 0x00, 0x00, 0xbc, 0x02, 0x00, 0x00, 0xb8, 0x02, 0x00, 0x00,
  0x18, 0x02, 0x00, 0x00, 0xcc, 0x01, 0x00, 0x00, 0x84, 0x01, 0x00, 0x00,
  0xa8, 0x02, 0x00, 0x00, 0x4c, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00,
  0x84, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00, 0x24, 0x01, 0x00, 0x00,
  0x00, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00, 0x0c, 0x00, 0x00, 0x00,
  0x08, 0x00, 0x0c, 0x00, 0x04, 0x00, 0x08, 0x00, 0x08, 0x00, 0x00, 0x00,
  0x08, 0x00, 0x00, 0x00, 0x06, 0x00, 0x00, 0x00, 0x13, 0x00, 0x00, 0x00,
  0x6d, 0x69, 0x6e, 0x5f, 0x72, 0x75, 0x6e, 0x74, 0x69, 0x6d, 0x65, 0x5f,
  0x76, 0x

## Transpose

### Define TensorFlow Model

In [13]:
@tf.function
def transpose(a, perm=None):
  return tf.transpose(a, perm=perm, name="TRANSPOSE")

### Transpose Examples

In [14]:
print("Transposes a, where a is a Tensor\n")

a = tf.constant([[1, 2, 3], [4, 5, 6]])
print("Number of dimensions:", a.ndim)
print("Shape of tensor:", a.shape)
print("Tensor:", a)

print("\n")

output_tensor = transpose(a)
print("Number of dimensions:", output_tensor.ndim)
print("Shape of tensor:", output_tensor.shape)
print("Tensor:", output_tensor)

print("\nPermutes the dimensions according to the value of perm\n")

a = tf.constant([[[ 1,  2,  3], [ 4,  5,  6]], [[ 7,  8,  9], [10, 11, 12]]])
perm = [0, 2, 1]

output_tensor = transpose(a, perm=perm)
print("Number of dimensions:", output_tensor.ndim)
print("Shape of tensor:", output_tensor.shape)
print("Tensor:", output_tensor)

Transposes a, where a is a Tensor

Number of dimensions: 2
Shape of tensor: (2, 3)
Tensor: tf.Tensor(
[[1 2 3]
 [4 5 6]], shape=(2, 3), dtype=int32)


Number of dimensions: 2
Shape of tensor: (3, 2)
Tensor: tf.Tensor(
[[1 4]
 [2 5]
 [3 6]], shape=(3, 2), dtype=int32)

Permutes the dimensions according to the value of perm

Number of dimensions: 3
Shape of tensor: (2, 3, 2)
Tensor: tf.Tensor(
[[[ 1  4]
  [ 2  5]
  [ 3  6]]

 [[ 7 10]
  [ 8 11]
  [ 9 12]]], shape=(2, 3, 2), dtype=int32)


### Convert to TensorFlow Lite for Microcontrollers Model

In [15]:
a = tf.constant([[[ 1,  2,  3], [ 4,  5,  6]], [[ 7,  8,  9], [10, 11, 12]]])
perm = [0, 2, 1]

function = transpose
tensors = [a, perm]
quantization = True
def representative_dataset():
     yield([a])

model_tflite_micro = generateCSourceFile(function, tensors, quantization, representative_dataset)

In [16]:
# Print the C source file
!cat {model_tflite_micro}

unsigned char g_model[] = {
  0x1c, 0x00, 0x00, 0x00, 0x54, 0x46, 0x4c, 0x33, 0x00, 0x00, 0x12, 0x00,
  0x1c, 0x00, 0x04, 0x00, 0x08, 0x00, 0x0c, 0x00, 0x10, 0x00, 0x14, 0x00,
  0x00, 0x00, 0x18, 0x00, 0x12, 0x00, 0x00, 0x00, 0x03, 0x00, 0x00, 0x00,
  0x38, 0x02, 0x00, 0x00, 0xe8, 0x00, 0x00, 0x00, 0xd0, 0x00, 0x00, 0x00,
  0x3c, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00,
  0x0c, 0x00, 0x00, 0x00, 0x08, 0x00, 0x0c, 0x00, 0x04, 0x00, 0x08, 0x00,
  0x08, 0x00, 0x00, 0x00, 0x08, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00,
  0x13, 0x00, 0x00, 0x00, 0x6d, 0x69, 0x6e, 0x5f, 0x72, 0x75, 0x6e, 0x74,
  0x69, 0x6d, 0x65, 0x5f, 0x76, 0x65, 0x72, 0x73, 0x69, 0x6f, 0x6e, 0x00,
  0x05, 0x00, 0x00, 0x00, 0x7c, 0x00, 0x00, 0x00, 0x68, 0x00, 0x00, 0x00,
  0x4c, 0x00, 0x00, 0x00, 0x30, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00,
  0xc6, 0xff, 0xff, 0xff, 0x04, 0x00, 0x00, 0x00, 0x10, 0x00, 0x00, 0x00,
  0x31, 0x2e, 0x36, 0x2e, 0x30, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00,
  0x00, 0x

## Space-To-Batch-ND

### Define TensorFlow Model

In [17]:
@tf.function
def space_to_batch_nd(tensor, block_shape, paddings):
  return tf.space_to_batch_nd(tensor, block_shape, paddings, name="SPACE_TO_BATCH_ND")

### Space-To-Batch-ND Examples

In [18]:
print("Divides spatial dimensions [1, ..., M] of the input into a grid of blocks of shape block_shape, and interleaves these blocks with the batch dimension (0) such that in the output, the spatial dimensions [1, ..., M] correspond to the position within the grid, and the batch dimension combines both the position within a spatial block and the original batch position.\n")
print("Prior to division into blocks, the spatial dimensions of the input are optionally zero padded according to paddings\n")

tensor = tf.constant([[[[1], [2]], [[3], [4]]]])
block_shape = np.array([2, 2])
paddings = np.array([[0, 0], [0, 0]])

print("Number of dimensions:", tensor.ndim)
print("Shape of tensor:", tensor.shape)
print("Tensor:", tensor)

print("\n")

output_tensor = space_to_batch_nd(tensor, block_shape, paddings)
print("Number of dimensions:", output_tensor.ndim)
print("Shape of tensor:", output_tensor.shape)
print("Tensor:", output_tensor)


Divides spatial dimensions [1, ..., M] of the input into a grid of blocks of shape block_shape, and interleaves these blocks with the batch dimension (0) such that in the output, the spatial dimensions [1, ..., M] correspond to the position within the grid, and the batch dimension combines both the position within a spatial block and the original batch position.

Prior to division into blocks, the spatial dimensions of the input are optionally zero padded according to paddings

Number of dimensions: 4
Shape of tensor: (1, 2, 2, 1)
Tensor: tf.Tensor(
[[[[1]
   [2]]

  [[3]
   [4]]]], shape=(1, 2, 2, 1), dtype=int32)


Number of dimensions: 4
Shape of tensor: (4, 1, 1, 1)
Tensor: tf.Tensor(
[[[[1]]]


 [[[2]]]


 [[[3]]]


 [[[4]]]], shape=(4, 1, 1, 1), dtype=int32)


### Convert to TensorFlow Lite for Microcontrollers Model

In [19]:
tensor = tf.constant([[[[1], [2]], [[3], [4]]]])
block_shape = np.array([2, 2])
paddings = np.array([[0, 0], [0, 0]])

function = space_to_batch_nd
tensors = [tensor, block_shape, paddings]
quantization = False

model_tflite_micro = generateCSourceFile(function, tensors, quantization, representative_dataset)

In [20]:
# Print the C source file
!cat {model_tflite_micro}

unsigned char g_model[] = {
  0x1c, 0x00, 0x00, 0x00, 0x54, 0x46, 0x4c, 0x33, 0x14, 0x00, 0x20, 0x00,
  0x04, 0x00, 0x08, 0x00, 0x0c, 0x00, 0x10, 0x00, 0x14, 0x00, 0x00, 0x00,
  0x18, 0x00, 0x1c, 0x00, 0x14, 0x00, 0x00, 0x00, 0x03, 0x00, 0x00, 0x00,
  0x3c, 0x00, 0x00, 0x00, 0x30, 0x00, 0x00, 0x00, 0x98, 0x00, 0x00, 0x00,
  0x0c, 0x00, 0x00, 0x00, 0x38, 0x00, 0x00, 0x00, 0x30, 0x00, 0x00, 0x00,
  0x06, 0x00, 0x00, 0x00, 0xc4, 0x02, 0x00, 0x00, 0xc0, 0x02, 0x00, 0x00,
  0xbc, 0x02, 0x00, 0x00, 0xb8, 0x02, 0x00, 0x00, 0xb4, 0x02, 0x00, 0x00,
  0x54, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00, 0x8c, 0x00, 0x00, 0x00,
  0x01, 0x00, 0x00, 0x00, 0x20, 0x01, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00,
  0x01, 0x00, 0x00, 0x00, 0x0c, 0x00, 0x00, 0x00, 0x08, 0x00, 0x0c, 0x00,
  0x04, 0x00, 0x08, 0x00, 0x08, 0x00, 0x00, 0x00, 0x08, 0x00, 0x00, 0x00,
  0x05, 0x00, 0x00, 0x00, 0x13, 0x00, 0x00, 0x00, 0x6d, 0x69, 0x6e, 0x5f,
  0x72, 0x75, 0x6e, 0x74, 0x69, 0x6d, 0x65, 0x5f, 0x76, 0x65, 0x72, 0x73,
  0x69, 0x